In [4]:
import os

In [5]:
%pwd

'c:\\Users\\ROSHAN\\OneDrive\\Desktop\\Vs Code\\Python\\Projects\\E2E\\research'

In [6]:
os.chdir('../')

In [7]:
%pwd

'c:\\Users\\ROSHAN\\OneDrive\\Desktop\\Vs Code\\Python\\Projects\\E2E'

Entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class  DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Configuration

In [9]:
from ml_project.constants import *

from ml_project.utils.common import read_yaml,create_directories

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH
                 ):
        
        self.config=read_yaml(config_filepath)
        self.schema=read_yaml(schema_filepath)       
        self.params=read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from ml_project import logger
from ml_project.utils.common import get_size

In [14]:
class DataIngestion:
    
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-18 13:54:21,296: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-18 13:54:21,300: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-18 13:54:21,303: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-18 13:54:21,327: INFO: common: created directory at: artifacts]
[2025-06-18 13:54:21,330: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-18 13:54:22,684: INFO: 707623563: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25115
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d5257e7a66c105f048c4082616b0a570d8819a7ce06583e19e8ae39748a829a4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 89AA:2D3750:10F1A3:26B8C3:685277B6
Accept-Ranges: bytes
Date: Wed, 18 